In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)


import os
from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.words import make_corpus_0, get_wordcount_list
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import inflect
from collections import Counter

#dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
#dic = load(dir_save)
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.87 µs


In [2]:
dic = load(dir_save = '../big_data/dic_20190830.pickle')
ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))
desc = [i for i in ls if len(dic[i]['description'])<1]
print('drop %d recipes with no description' %(len(desc)))
print('now we are using recipe54k %d' % len(ls))

exist
drop 46 recipes with less than 2 ingredients
furthur drop 1026 recipes with less than 2 instructions
drop 0 recipes with no description
now we are using recipe54k 54076
time: 2.29 s


In [236]:
dir_HugeFiles

'/data/yueliu/RecipeAnalytics_201906/'

time: 30.8 ms


In [237]:
df = pd.read_csv('../big_data/food_taxonomy.txt',delimiter='\t', header = None)

time: 37.1 ms


In [4]:
def cleaning(df):
    '''
    eliminate the row if it contains the following non-ingredient words
    '''
    print('origin: length of data: %d' % len(df))
    eliminate = ['Snack brand', 'Preparation', 'Fast food', 'Dietary Supplement', 'Dessert']
    for i in range(2):
        df  = df[df.apply(lambda x: x[i] not in eliminate, axis = 1)]
    # uncased
    for i in range(3):
        df[i]  = df.apply(lambda x: x[i].lower(), axis = 1)
    print('drop some fields: length of data: %d' % len(df))    
    
    # drop the duplicates
    df = df[~df[2].duplicated()]
    df = df.reset_index(drop = True)
    print('drop duplicates: length of data: %d' % len(df))
        
    return df
df = cleaning(df)

origin: length of data: 4294
drop some fields: length of data: 2682
drop duplicates: length of data: 2582
time: 829 ms


In [5]:
for i in range(3):
    print(df[i].unique())

['bean and legume' 'beverage' 'breakfast cereal' 'condiment'
 'egg and dairy' 'fruit' 'herb and spice' 'meat' 'mushroom' 'nut and seed'
 'side dish' 'snack' 'staple' 'vegetable']
['legume' 'soy product' 'alcohol' 'chocolate' 'coffee' 'energy drink'
 'fruit juice' 'hot beverage' 'lemon beverage' 'milk substitutes'
 'rice beverage' 'soft drink' 'breakfast cereal' 'condiment'
 'dairy product' 'egg' 'berry' 'mediterranean' 'temperate' 'tropical'
 'herbs' 'mixtures' 'peppers' 'spices' 'beef' 'fish' 'game' 'lamb' 'meat'
 'meatball' 'pork' 'poultry' 'sausage' 'seafood' 'shellfish' 'mushroom'
 'gymnosperm seed' 'nut' 'other' 'pseudocereal' 'antipasto' 'fries'
 'legumes' 'maize' 'pasta' 'potatoes' 'salad' 'soup' 'snack' 'banana'
 'other cereal' 'rice' 'root and tuber' 'wheat' 'bulb and stem vegetables'
 'flowers and flower buds' 'fruits' 'leafy and salad' 'podded vegetables'
 'root and tuberous vegetables' 'sea vegetables' 'vegetables']
['bean' 'chickpea' 'cowpea' ... 'vegetables' 'vegetarian' 

In [6]:
additional_ingr=\
[
['Condiment', 'Sweet', 'sugar'],
['Condiment', 'Sweet', 'applesauce'],
['Flour', 'Flour','flour'],
['Baking powder','Baking powder','baking powder'],
['Water','Water','water'],
['Water','Water','iced'],
['Water','Water','ice'],
['Herb and spice','spices','jalapeno'],
['Condiment','Condiment', 'oil'],
['Beverage','Fruit juice', 'juice'],
['Staple','Maize','enchilada'],
['Condiment','Condiment', 'dressing mix'],
['Condiment','Condiment', 'cheese mix'],
['Baking powder','Baking powder','baking soda'],
['Condiment','Condiment', 'dry mix'],
['Condiment','Condiment','chocolate'],
['Vegetable','Bulb and stem vegetables','tapioca'],
['Flour', 'Flour','xanthan gum'],
['Flour', 'Flour','starch'],
['Egg and dairy', 'Dairy product','buttermilk'],
['Condiment','Condiment','chili'],
['Condiment','Condiment','chile'],
['Condiment','Condiment','chilis'],
['Condiment','Condiment','chiles'],
['Condiment','Condiment','chilies'],
['Flour', 'Flour','corn muffin mix'],
['Beverage','Chocolate','chocolate mix'],
['Meat','dumpling','dumpling'],
['Meat','dumpling','wonton'],
['Staple','Wheat','pizza dough'],
['Staple','Wheat','dough'],
['Condiment','Condiment', 'pizza sauce'],
['Flour', 'Flour','yeast'],
['Condiment','Sweet','cocoa'],
['Staple','Maize','chip'],
['Egg and dairy','Dairy product','ricotta'],
['Condiment','Condiment','seasoning'],
['Beverage','Alcohol','sherry'],
['Staple','Rice','grain rice'], 
['Staple','Wheat','shell'],
['Meat','Beef','fillet'],
['Staple','Maize','cornmeal'],
['Condiment','Condiment','seed oil'],
['Nut and seed','Other','seed'],
['Condiment', 'Sweet', 'sugar blend'],
['Soup','Soup','broth'],
['Soup','Soup','stock'],
['Condiment', 'Sweet', 'marshmallow'],
['Condiment', 'Dry Condiment', 'dried vegetable flakes'],
['Condiment', 'Dry Condiment', 'dried celery flakes'],
['Flour', 'Flour','cornstarch'],
['Staple','Wheat','double crust'],
['Staple','Wheat','crust'],
['Staple','Wheat','pastry crust'],
['Egg and dairy','Dairy product','gorgonzola'],
['Beverage','juice','drink mix'],
['Egg and dairy','Egg','Egg whites'],
['Baking powder','Baking powder','baking mix'],
['Staple','Rice','brown rice'],
['Condiment','Condiment','five spice'],
['Meat','Beef','tenderloin'],
['Meat','Pork','prosciutto'],
['Condiment', 'Sweet', 'whipped topping'],
['Condiment', 'Sweet', 'topping'],
['Beverage','Alcohol','cider'],
['Meat','Shellfish','crabmeat'],
['Condiment', 'Sweet', 'candy'],
['Condiment', 'Sweet', 'caramel'],
['Condiment', 'Sweet', 'molasses'],
['Vegetable','Podded vegetables','cannellini'],
['Vegetable','Fruits','fruit'],
['Staple','Wheat','saltine'],
['Condiment','Condiment', 'habanero'],
['Beverage','Juice','jell o'],
['Beverage','Juice','jelly'],
['Beverage','Soft drink','carbonated beverage'],
['Egg and dairy','Dairy product','gruyere'],
['Vegetable','Leafy and Salad','beet'],
['Water','Water','icing'],
['Egg and dairy','Dairy product','parmigiano'],
['Beverage','Alcohol','liqueur'],
['Condiment','Condiment', 'lard'],
['Staple','Wheat','crumb'],
['Herb and spice','Herb','peppermint'],
['Beverage','Alcohol','marsala'],
['Side dish','Potatoes','hash brown'],
['Meat','Beef','steak'],
['Condiment','Condiment','gelatin'],
['Meat','Beef','chuck'],
['Egg and dairy','Dairy product','colby'],
['Condiment', 'Sweet', 'jam'],
['Condiment', 'Sweet', 'cool whip'],
['Condiment', 'Sweet', 'stevia'],
['Staple','Wheat','bran'],
['Condiment','Condiment','pimento'],
['Condiment','Condiment','food coloring'],
['Meat','Meat','rib'],
['Condiment','Condiment','shortening'],
['Vegetable','Fruits','sweet pickles'],
['Condiment', 'Sweet', 'white confectioner'],
['Condiment', 'Sweet', 'confectioner'],  
['Vegetable','Root and tuberous vegetabless','rhubarb'],
['Condiment', 'Condiment', 'cooking spray']
]

def add_rows(df, additional_ingr):
    add = pd.DataFrame(additional_ingr)
    print('before processing: length of data: %d' % len(df))
    df = pd.concat([df,add]).reset_index(drop =True)

    # uncased
    for i in range(3):
        df[i]  = df.apply(lambda x: x[i].lower().strip(), axis = 1)
    print('add some rows: length of data: %d' % len(df))    
    
    # drop the duplicates
    df = df[~df[2].duplicated()]
    df = df.reset_index(drop = True)
    print('drop duplicates: length of data: %d' % len(df))
    return df

df = add_rows(df, additional_ingr)

before processing: length of data: 2582
add some rows: length of data: 2685
drop duplicates: length of data: 2684
time: 472 ms


In [10]:
df.head(1)

,0,1,2
0,bean and legume,legume,bean


time: 33 ms


In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [148]:
def exact_match(ny_ingred):
    '''
    Args: ingred: a str
    Return: str: the first exact match or no change
    '''
    assert type(ny_ingred) == str
    phrases_to_sentences = 'Mix the %s and water.'%(ny_ingred)
    doc = nlp(phrases_to_sentences)
    exact_match, root_match = [],[]
    for chunk in doc.noun_chunks:
        if chunk.text != 'water':
            root_lemma = [token.lemma_ for token in doc if token.text == chunk.root.text][0]
            exact_match.append(chunk.lemma_.replace('the ',''))
            root_match.append(root_lemma)
    if exact_match:
        return exact_match[0]
    else:
        return ny_ingred

time: 35.9 ms


In [149]:
### add root
df[3] = df[2].apply(lambda x: exact_match(x))

time: 27.5 s


In [40]:
dic = load(dir_save = '../big_data/dic_regional_20190925.pickle')

ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))
desc = [i for i in ls if len(dic[i]['description'])<1]
print('drop %d recipes with no description' %(len(desc)))
print('now we are using recipe54k %d' % len(ls))

exist
drop 46 recipes with less than 2 ingredients
furthur drop 1026 recipes with less than 2 instructions
drop 0 recipes with no description
now we are using recipe54k 54076
time: 8.57 s


In [151]:
### 
level3 = df.groupby([3])[1].apply(list).index.tolist()
gb = df.groupby([3])[1].apply(list)
level3_set = [set(ingr.split(' ')) for ingr in level3]

time: 916 ms


In [112]:
import random 
random.seed(2019)

time: 26.2 ms


In [ ]:
random.sample(matched, 1)[0]

In [187]:
exact = dic[9]['ny_ingredients']['exact']
def delexicalized(exact_ingr):
    '''
    input: dic[9]['ny_ingredients']['exact'][0]
    e.g. 'onion'
    '''
    matched = []
    for ingr, ingr_set in zip(level3, level3_set):
        if len(ingr_set) == len(ingr_set & set(exact_ingr.split(' '))):
            matched += gb[ingr]
    if matched:
        return '%s#%s' % (random.sample(matched, 1)[0], exact_ingr)

time: 31.4 ms


In [165]:
[delexicalized(x) for x in dic[9]['ny_ingredients']['exact']]

['poultry#turkey',
 'bulb and stem vegetables#celery',
 'bulb and stem vegetables#onion',
 'tropical#pecan',
 'condiment#salt',
 'condiment#mayonnaise',
 'mediterranean#lemon juice',
 'dairy product#cheddar cheese',
 'maize#potato chip']

time: 44.3 ms


In [211]:
def delexicalized_sent(sent):
    '''
    For example
    sent = 'Mix the porks, chicken, white sugar with the grain rice in a pot.'
    delexicalized_sent(sent) = 
    'Mix the pork#pork, poultry#chicken, sweet#white sugar with the rice#grain rice in a pot.'
    '''
    output = sent
    doc = nlp(sent)
    for chunk in doc.noun_chunks:
        food_matching = delexicalized(chunk.lemma_.replace('the ',''))
        if food_matching:
            output = output.replace(chunk.text.replace('the ',''), food_matching)
    return output

time: 29.3 ms


In [210]:
delexicalized_sent('Mix the porks, chicken, white sugar with the grain rice in a pot.')

'Mix the pork#pork, poultry#chicken, sweet#white sugar with the rice#grain rice in a pot.'

time: 55.9 ms


In [225]:
import random
import spacy
class food_taxonomy:
    def __init__(self, df):
        ### 
        self.level3 = df.groupby([3])[1].apply(list).index.tolist()
        self.gb = df.groupby([3])[1].apply(list)
        self.level3_set = [set(ingr.split(' ')) for ingr in level3]
        self.random = random
        self.random.seed(2019)
        self.spacy = spacy.load('en_core_web_lg')
        
    def delexicalized(self, exact_ingr):
        '''
        input: dic[9]['ny_ingredients']['exact'][0]
        e.g. 'onion'
        '''
        matched = []
        for ingr, ingr_set in zip(self.level3, self.level3_set):
            if len(ingr_set) == len(ingr_set & set(exact_ingr.split(' '))):
                matched += self.gb[ingr]
        if matched:
            return '%s#%s' % (self.random.sample(matched, 1)[0], exact_ingr)
    
    def delexicalized_sent(self, sent):
        '''
        For example
        sent = 'Mix the porks, chicken, white sugar with the grain rice in a pot.'
        delexicalized_sent(sent) = 
        'Mix the pork#pork, poultry#chicken, sweet#white sugar with the rice#grain rice in a pot.'
        '''
        output = sent
        doc = self.spacy(sent)
        for chunk in doc.noun_chunks:
            food_matching = self.delexicalized(chunk.lemma_.replace('the ',''))
            if food_matching:
                output = output.replace(chunk.text.replace('the ',''), food_matching)
        return output

time: 36.8 ms


In [227]:
model = food_taxonomy(df)

time: 16.1 s


In [234]:
model.delexicalized_sent('Mix the porks, chicken, white sugar with the grain rice in a pot.')

'Mix the pork#pork, poultry#chicken, sweet#white sugar with the spices#grain rice in a pot.'

time: 61.7 ms


In [240]:
dic[388]['name']

['tater tot taco casserole']

time: 28.6 ms
